In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision
import numpy as np

import resnet

### load data

In [3]:
norm_mean = 0
norm_var = 1
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((norm_mean,norm_mean,norm_mean), (norm_var, norm_var, norm_var)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((norm_mean,norm_mean,norm_mean), (norm_var, norm_var, norm_var)),
])
cifar_train = datasets.CIFAR10("./data", train=True, download=True, transform=transform_train)
cifar_test = datasets.CIFAR10("./data", train=False, download=True, transform=transform_test)
train_loader = DataLoader(cifar_train, batch_size = 256, shuffle=True)
test_loader = DataLoader(cifar_test, batch_size = 100, shuffle=True)


0it [00:00, ?it/s]


100%|█████████▉| 169820160/170498071 [00:07<00:00, 21975442.74it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### load model

In [9]:
gamma = 0.2
eps = 0.031
filepath = f'spresnet18_{gamma}_cifar_adv.pth'
device = torch.device('cuda:0')

model = resnet.SparseResNet18(sparsities=[gamma, gamma, gamma, gamma], sparse_func='vol').to(device)
model.load_state_dict(torch.load(filepath))
model.eval();

In [11]:
acc = 0
total_number = 0

for images, labels in test_loader:
    logits = model(images.to(device))
    acc += np.sum(logits.detach().cpu().numpy().argmax(1) == labels.cpu().numpy())
    total_number += images.shape[0]

print(acc / total_number)

0.7693
